In [ ]:
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader,Dataset

from training.engine import *
from training.model import RNNModel
from training.config import *
from training.dataset import *

device = torch.device("cuda:1")

import warnings
warnings.filterwarnings("ignore")

In [ ]:
grouped = pd.read_csv("./data/SAB_Shunt.csv")

In [ ]:
config = TrainGlobalConfigShunt_with_all()

In [ ]:
var = config.target

from sklearn.model_selection import StratifiedKFold
X = grouped["Fallnummer"].values
y = grouped[var]

skf = StratifiedKFold(n_splits=6, random_state = 1)

grouped["fold"] = 0

for idx, (train_index, test_index) in enumerate(skf.split(X, y)):
    grouped["fold"][grouped["Fallnummer"].isin(X[test_index])] = idx

In [ ]:
model = RNNModel(input_dim = len(config.features)).to(device)

In [ ]:
results = []
results_all = []

for val_fold in range(0,6):
    for test_fold in range(0,6):
        if(val_fold != test_fold):
            
            model = RNNModel(input_dim = len(config.features)).to(device)

            train_dataset = ICUDataset(df = grouped, 
                                       pat_ids = grouped["Fallnummer"][(grouped["fold"] != val_fold) & (grouped["fold"] != test_fold)].unique(), 
                                       config = config,
                                       mode = "train", 
                                     )
            val_dataset = ICUDataset(df = grouped, 
                                     pat_ids = grouped["Fallnummer"][(grouped["fold"] == val_fold)].unique(), 
                                     config = config,
                                     mode = "val")

            train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers = config.num_workers)
            val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers = config.num_workers)

            fitter = Fitter(model, device, config)
            fitter.fit(train_loader, val_loader, str(val_fold) + "_"+ str(test_fold))